In [1]:
import matplotlib.pyplot as plt
from maploc.demo import Demo, read_input_image

# Increasing the number of rotations increases the accuracy but requires more GPU memory.
# The highest accuracy is achieved with num_rotations=360
# but num_rotations=64~128 is often sufficient.
# To reduce the memory usage, we can reduce the tile size in the next cell.
demo = Demo(num_rotations=64, device='cuda')

Loaded as API: https://jinlinyi-perspectivefields.hf.space/ ✔


/root/.virtualenvs/OrienterNet/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/root/.virtualenvs/OrienterNet/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/root/.virtualenvs/OrienterNet/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/root/.virtualenvs/OrienterNet/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


In [3]:
# no EXIF data: provide a coarse location prior as address
image_path = "assets/52.14046098_11.648074903333333.png"
prior_address = "Pfälzer Platz Magdeburg"

# Try out these other queries!
# image_path = "assets/query_vancouver_1.JPG"
# prior_address = "Vancouver Waterfront Station"

# image_path = "assets/query_vancouver_2.JPG"
# image_path = "assets/query_vancouver_3.JPG"
# prior_address = None # here we load the location prior from the exif

image, camera, gravity, proj, bbox, prior_latlon = read_input_image(
    image_path,
    prior_address=prior_address,
    tile_size_meters=128,  # try 64, 256, etc.
)

# Show the query area in an interactive map
from maploc.osm.viz import GeoPlotter
# plot = GeoPlotter(zoom=16)
# plot.points(prior_latlon[:2], "red", name="location prior", size=10)
# plot.bbox(proj.unproject(bbox), "blue", name="map tile")
# plot.fig.show("notebook")

# Query OpenStreetMap for this area
from maploc.osm.tiling import TileManager
tiler = TileManager.from_bbox(proj, bbox + 10, demo.config.data.pixel_per_meter)
canvas = tiler.query(bbox)

# Show the inputs to the model: image and raster map
from maploc.osm.viz import Colormap, plot_nodes
from maploc.utils.viz_2d import plot_images
map_viz = Colormap.apply(canvas.raster)
# plot_images([image, map_viz], titles=["input image", "OpenStreetMap raster"])
# plot_nodes(1, canvas.raster[2], fontsize=6, size=10)

[2023-11-16 16:57:28 maploc INFO] Using prior address 'Pfälzer Platz, Hohepfortestraße, Alte Neustadt, Magdeburg, Sachsen-Anhalt, 39106, Deutschland'
[2023-11-16 16:57:28 maploc INFO] Calling the PerspectiveFields calibrator, this may take some time.
[2023-11-16 16:57:39 maploc INFO] Using (roll, pitch) (1.2, -3.76).
[2023-11-16 16:57:39 maploc INFO] Calling the OpenStreetMap API...


In [8]:
from maploc.utils.viz_localization import (
    likelihood_overlay,
    plot_dense_rotations,
    add_circle_inset,
)
from maploc.utils.viz_2d import features_to_RGB

# Run the inference
uv, yaw, prob, neural_map, image_rectified = demo.localize(image, camera, canvas, roll_pitch=gravity)
print(uv.shape, yaw.shape, prob.shape, neural_map.shape, image_rectified.shape)
print(uv, yaw)

# Visualize the predictions
# overlay = likelihood_overlay(prob.numpy().max(-1), map_viz.mean(-1, keepdims=True))
# (neural_map_rgb,) = features_to_RGB(neural_map.numpy())
# plot_images([overlay, neural_map_rgb], titles=["prediction", "neural map"])
# ax = plt.gcf().axes[0]
# ax.scatter(*canvas.to_uv(bbox.center), s=5, c="red")
# plot_dense_rotations(ax, prob, w=0.005, s=1/25)
# add_circle_inset(ax, uv)
# plt.show("notebook")

# Plot as interactive figure
bbox_latlon = proj.unproject(canvas.bbox)
print(bbox_latlon)
plot = GeoPlotter(zoom=16.5)
#plot.raster(map_viz, bbox_latlon, opacity=0.5)
# plot.raster(likelihood_overlay(prob.numpy().max(-1)), proj.unproject(bbox))
plot.points(prior_latlon[:2], "red", name="location prior", size=10)
plot.points(proj.unproject(canvas.to_xy(uv)), "black", name="argmax", size=16)
# plot.bbox(bbox_latlon, "blue", name="map tile")
plot.fig.show()

torch.Size([2]) torch.Size([]) torch.Size([512, 512, 64]) torch.Size([8, 512, 512]) torch.Size([3, 512, 864])
tensor([271., 442.]) tensor(247.5000)
52.13993648041176,11.64595606284107,52.142237189725456,11.649695368913001
